In [1]:
import sys  
from os import path
import numpy as np
import pandas as pd

sys.path.append('../src/')

In [2]:
# Load initial node data from file:
from modules.classes.Data_Manipulator import Data_Manipulator

file_dir = "D:/Igor/Research_USF/University of South Florida/Mao, Wenbin - Igor/Febio-Models/Active-Models/PAQ/Myo_test/myo_hex_coarse_8_PAQ"
file_name = "myo_hex_coarse_8_PAQ.feb"

file_path = path.join(file_dir, file_name)

data_m = Data_Manipulator(file_path)
node_data = data_m.get_node_data_from_face(data_m.face_dicts[0])

xyz = node_data[["x","y","z"]].to_numpy()

# Compute volume using Convex Hull

In [3]:
from scipy.spatial import ConvexHull

vol_ch = ConvexHull(xyz, qhull_options="Qt Qx Qv Q4 Q14").volume


In [4]:
#  ref: https://stackoverflow.com/questions/24733185/volume-of-convex-hull-with-qhull-from-scipy
def tetrahedron_volume(a, b, c, d):
    return np.abs(np.einsum('ij,ij->i', a-d, np.cross(b-d, c-d))) / 6

def convex_hull_volume_bis(pts):
    ch = ConvexHull(pts)
    simplices = np.column_stack((np.repeat(ch.vertices[0], ch.nsimplex), ch.simplices))
    tets = ch.points[simplices]
    return np.sum(tetrahedron_volume(tets[:, 0], tets[:, 1], tets[:, 2], tets[:, 3]))

In [5]:

convex_hull_volume_bis(xyz)

122754.87266821964

In [6]:
vol_ch

122754.87266821973

# Compute Exact value

In [7]:
a = 65
b = 25
pi = np.pi

m = 20
n = -65

vol = pi * b*b * (m - n - ( (m**3 - n**3)/(3* a**2) ))
vol

123115.42216175977

In [8]:
vol_ch - vol

-360.54949354003475

# Compute based on tetrahedrons

In [9]:
surfs = data_m.get_nodes_from_surface([("Surface", "Endocardio", "any")])
endo_surf = surfs[0]

In [10]:
centroid = node_data[["x", "y", "z"]].mean().to_numpy()

In [11]:
centroid

array([-6.39232769e-17, -1.96919206e-16, -2.52217891e+01])

## Using an approximation method 

In [12]:
def comp_tri_area(coords):
    v = coords[0] - coords[1]
    w = coords[0] - coords[2]
    return np.linalg.norm(0.5 * np.cross(v,w))

In [13]:
#ref https://stackoverflow.com/questions/12642256/python-find-area-of-polygon-from-xyz-coordinates
#determinant of matrix a
def det(a):
    return a[0][0]*a[1][1]*a[2][2] + a[0][1]*a[1][2]*a[2][0] + a[0][2]*a[1][0]*a[2][1] - a[0][2]*a[1][1]*a[2][0] - a[0][1]*a[1][0]*a[2][2] - a[0][0]*a[1][2]*a[2][1]

#unit normal vector of plane defined by points a, b, and c
def unit_normal(a, b, c):
    x = det([[1,a[1],a[2]],
             [1,b[1],b[2]],
             [1,c[1],c[2]]])
    y = det([[a[0],1,a[2]],
             [b[0],1,b[2]],
             [c[0],1,c[2]]])
    z = det([[a[0],a[1],1],
             [b[0],b[1],1],
             [c[0],c[1],1]])
    magnitude = (x**2 + y**2 + z**2)**.5
    return (x/magnitude, y/magnitude, z/magnitude)

#dot product of vectors a and b
def dot(a, b):
    return a[0]*b[0] + a[1]*b[1] + a[2]*b[2]

#cross product of vectors a and b
def cross(a, b):
    x = a[1] * b[2] - a[2] * b[1]
    y = a[2] * b[0] - a[0] * b[2]
    z = a[0] * b[1] - a[1] * b[0]
    return (x, y, z)

#area of polygon poly
def area(poly):
    if len(poly) < 3: # not a plane - no area
        return 0

    total = [0, 0, 0]
    for i in range(len(poly)):
        vi1 = poly[i]
        if i is len(poly)-1:
            vi2 = poly[0]
        else:
            vi2 = poly[i+1]
        prod = cross(vi1, vi2)
        total[0] += prod[0]
        total[1] += prod[1]
        total[2] += prod[2]
    result = dot(total, unit_normal(poly[0], poly[1], poly[2]))
    return abs(result/2)

In [14]:
def get_centr(vertices):
    return vertices.transpose().mean(axis=1)

def get_unit_vector(vec):
    return vec / np.linalg.norm(vec)

def get_angle_between(v1, v2):
    v1_u = get_unit_vector(v1)
    v2_u = get_unit_vector(v2)

    angle = np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0))
    delta = v1[0]*v2[1] - v1[1]*v2[0]
    if delta > 0:
        angle += np.pi

    return angle
    
def sort_by_internal_angle(vertices):
    # get centroid
    centr = get_centr(vertices)
    v_an = list()
    v1 = centr - vertices[0]
    for p2 in vertices[1:]:
        v2 = centr - p2
        v_an.append( (get_angle_between(v1,v2), p2) )
    # print(v_an)
    arr = np.array(v_an, dtype=[('a', float), ('v',object)])
    # print(arr[:3])
    s_arr = np.sort(arr, order=['a'])
    return np.vstack((vertices[0], [ve[1] for ve in s_arr]))

def normal(v1,v2):
    return np.cross(v1, v2)

def poly_area(vertices):
    """Input: np array of vertices """
    # ref http://geomalgorithms.com/a01-_area.html#3D%20Polygons
    
    # check if it is a polygon:
    if len(vertices) < 3:
        return 0.0
    vertices = np.array(vertices)
    # Check if polygon is closed:
    # if vertices[0].all() != vertices[-1].all():
    

    # sort vertices
    vertices = sort_by_internal_angle(vertices)
    # # print(vertices.reverse())

    vertices = vertices.tolist()
    vertices.reverse()
    if vertices[0] != vertices[-1]: 
        vertices = np.vstack((vertices, vertices[0]))

    vertices = np.array(vertices)

    # get number of vertices
    n = len(vertices) - 1

    # get plane normal vector
    A1 = vertices[0] - vertices[1]
    A2 = vertices[0] - vertices[2]

    normal_vec = np.cross(A1, A2)
    # use abs coord to ignore for projection
    abs_normal_vec = np.absolute(normal_vec)
    # select largest coord
    coord = 3;
    if abs_normal_vec[0] > abs_normal_vec[1]:
        if abs_normal_vec[0] > abs_normal_vec[2]:
            coord = 1
    elif abs_normal_vec[1] > abs_normal_vec[2]:
        coord = 2
    

    # scale
    an = np.linalg.norm(abs_normal_vec)
    
    # compute area of 2D polygon
    area = 0.0
    j = 2
    k = 0

    if coord == 1:
        for (i, j, k) in zip(range(1, n), range(2, n), range(0, n)):
            area += vertices[i][1] * (vertices[j][2] - vertices[k][2])
    elif coord == 2:
        for (i, j, k) in zip(range(1, n), range(2, n), range(0, n)):
            area += vertices[i][2] * (vertices[j][0] - vertices[k][2])
    elif coord == 3:
        for (i, j, k) in zip(range(1, n), range(2, n), range(0, n)):
            area += vertices[i][0] * (vertices[j][1] - vertices[k][1])

    # wrap around
    if coord == 1:
        area += vertices[n][1] * (vertices[1][2] - vertices[n-1][2])
        area *= (an/ (2 * normal_vec[0]))
    if coord == 2:
        area += vertices[n][2] * (vertices[1][0] - vertices[n-1][0])
        area *= (an/ (2 * normal_vec[1]))
    if coord == 3:
        area += vertices[n][0] * (vertices[1][1] - vertices[n-1][1])
        area *= (an/ (2 * normal_vec[2]))

    return area


### Validating algorithm

In [15]:
# V1 = [0, 0, 0]
# V2 = [1, 0, 0]
# V3 = [0, 1, 0]
# V4 = [1, 5, 0]

V1 = [2, 3, 0]
V2 = [6, 9, 0]
V3 = [15, 12, 0]
V4 = [16, 2, 0]

vertices = np.array([V1, V4, V3, V2])

poly_area(vertices)

97.5

### Tryig to compute volume

In [16]:
from scipy.spatial import Delaunay

volume = 0
for key in endo_surf:
    c = endo_surf[key]
    nodes = [v - 1 for v in c]
    data = node_data.loc[nodes]
    _centr = data[["x", "y", "z"]].mean().to_numpy()
    _xyz = data[["x", "y", "z"]].to_numpy()

    a = poly_area(_xyz)
    h = np.linalg.norm(centroid - _centr)

    volume += (1/3) * h * a

In [17]:
vol - volume

332854.07889173983

### Using an implementation found online

In [18]:
# ref https://stackoverflow.com/questions/12642256/python-find-area-of-polygon-from-xyz-coordinates

def check(p1, p2, base_array):
    idxs = np.indices(base_array.shape) # Create 3D array of indices

    p1 = p1.astype(float)
    p2 = p2.astype(float)

    # Calculate max column idx for each row idx based on interpolated line between two points
    max_col_idx = (idxs[0] - p1[0]) / (p2[0] - p1[0]) * (p2[1] - p1[1]) +  p1[1]    
    sign = np.sign(p2[0] - p1[0])
    return idxs[1] * sign <= max_col_idx * sign

def create_polygon(vertices):
    base_array = np.zeros(vertices.shape, dtype=float) 
    fill = np.ones(base_array.shape) * True 

    for k in range(vertices.shape[0]):
        fill = np.all([fill, check(vertices[k-1], vertices[k], base_array)], axis=0)

    base_array[fill] = 1

    return base_array

In [19]:
from scipy.spatial import Delaunay

volume = 0
for key in endo_surf:
    c = endo_surf[key]
    nodes = [v - 1 for v in c]
    data = node_data.loc[nodes]
    _centr = data[["x", "y", "z"]].mean().to_numpy()
    _xyz = data[["x", "y", "z"]].to_numpy()
    # while len(_xyz) < 5:
    #     _xyz = np.vstack((_xyz, _xyz[np.random.randint(len(_xyz))]))
    # print(_xyz)
    # tri = Delaunay(_xyz,qhull_options="Qt")
    # a = np.sum([comp_tri_area(s) for s in tri.simplices])
    a = area(_xyz)
    h = np.linalg.norm(centroid - _centr)

    volume += (1/3) * h * a



In [20]:
vol - volume

746.8007476855273